### Data Preparation

In [1]:
import os
import pandas as pd
from PIL import Image,UnidentifiedImageError
import numpy as np

# Path to the folder containing subfolders for each day
image_folder = './dataset/006_Banqiao'

# Load numerical data
numerical_data = pd.read_csv('./dataset/Banqiao_2022.csv')
#drop the Station column
numerical_data = numerical_data.drop(columns=['Station'])
# Ensure the 'date' column in numerical_data is in datetime format
print(numerical_data.head())
numerical_data['date'] = pd.to_datetime(numerical_data['date'],format="%d-%m-%Y %H:%M",dayfirst=True)

# Function to load images for a specific date
def load_images_for_date(date):
    date_str = date.strftime('%Y%m%d')
    images = []
    for hour in range(24):
        img_name=f"006-{date_str}{hour:02d}00.jpg"
        img_path = os.path.join(image_folder, date_str, img_name)
        try: 
            if os.path.exists(img_path):
                with Image.open(img_path) as img:
                    images.append(img.copy())
            else:
                images.append(None)
        except(OSError,UnidentifiedImageError):
            images.append(None)
    return images

# Create a dictionary to store images by date
image_data = {date: load_images_for_date(date) for date in numerical_data['date'].dt.date.unique()}


               date measurement     0     1     2     3     4     5     6  \
0  01-02-2022 00:00    AMB_TEMP    16    16    16  16.4  16.4  16.7  16.8   
1  01-02-2022 00:00         CH4     2  2.02  2.05  2.04  2.07  2.09  2.08   
2  01-02-2022 00:00          CO  0.24  0.25  0.23   0.2   0.2  0.21  0.21   
3  01-02-2022 00:00        NMHC  0.03  0.04  0.02  0.02  0.03  0.03  0.02   
4  01-02-2022 00:00          NO   0.2   0.6   0.6   0.6   0.5   0.6   0.5   

      7  ...    14    15    16    17    18    19    20    21    22    23  
0  16.9  ...    17  16.6  16.3  16.3  16.3  16.1  16.2  16.5  16.5  16.4  
1  2.11  ...  2.14  2.09  2.04  2.04  2.08  2.09  2.08  2.04  2.01  1.99  
2  0.28  ...  0.51  0.41   0.3  0.34  0.41  0.34  0.35  0.28  0.24  0.22  
3  0.05  ...  0.14   0.1  0.05  0.07  0.12  0.07  0.09  0.04  0.02     0  
4     1  ...   2.7   1.8   1.1   0.9   4.6   0.8   0.8   0.9   0.5   0.5  

[5 rows x 26 columns]


In [2]:
print(numerical_data)

           date measurement     0     1     2     3     4     5     6     7  \
0    2022-02-01    AMB_TEMP    16    16    16  16.4  16.4  16.7  16.8  16.9   
1    2022-02-01         CH4     2  2.02  2.05  2.04  2.07  2.09  2.08  2.11   
2    2022-02-01          CO  0.24  0.25  0.23   0.2   0.2  0.21  0.21  0.28   
3    2022-02-01        NMHC  0.03  0.04  0.02  0.02  0.03  0.03  0.02  0.05   
4    2022-02-01          NO   0.2   0.6   0.6   0.6   0.5   0.6   0.5     1   
...         ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   
1057 2022-03-31         THC  2.11  2.03  1.98  1.99  1.98  2.07     2  2.04   
1058 2022-03-31       WD_HR    58    86    78    73    91    70   112    64   
1059 2022-03-31  WIND_DIREC    62    98    73    97    77    31   103    60   
1060 2022-03-31  WIND_SPEED   1.3   2.1   2.7     2   1.7   0.6   2.2   1.3   
1061 2022-03-31       WS_HR   1.1   1.6   1.9   1.6   1.3   0.8   1.7   1.5   

      ...    14    15    16    17    18    19    20

### Data Preprocessing

In [3]:
import torchvision.transforms as transforms
from torchvision.models import resnet50,ResNet50_Weights,resnet152,ResNet152_Weights
import torch

# Preprocess images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load pretrained ResNet model
resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final classification layer

# Extract features from images
def extract_image_features(images):
    features = []
    for img in images:
        if img is not None:
            img = transform(img).unsqueeze(0)  # Add batch dimension
            with torch.no_grad():
                feature = resnet(img).squeeze()  # Remove batch dimension
            features.append(feature.numpy())
        else:
            features.append(np.zeros(2048))  # Handle missing images
    return features

# Extract image features for each date
image_features = {date: extract_image_features(images) for date, images in image_data.items()}
#free up space
del resnet

c:\Users\LENOVO\anaconda3\envs\teep_test1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(image_features)

{datetime.date(2022, 2, 1): [array([0.0670068 , 0.12476244, 0.04962633, ..., 0.12950025, 0.01728779,
       0.00068228], dtype=float32), array([0.03994673, 0.11152672, 0.04004784, ..., 0.11800665, 0.10802081,
       0.01046805], dtype=float32), array([0.04155901, 0.09191589, 0.1039572 , ..., 0.08011416, 0.07610244,
       0.03191404], dtype=float32), array([0.04078778, 0.15535934, 0.11373422, ..., 0.04260029, 0.09667775,
       0.10086785], dtype=float32), array([0.00314758, 0.1830483 , 0.07763058, ..., 0.03743078, 0.10852514,
       0.08504075], dtype=float32), array([0.07174712, 0.1762413 , 0.12477986, ..., 0.04017758, 0.01381355,
       0.0205706 ], dtype=float32), array([0.11655023, 0.14859438, 0.12377601, ..., 0.1744964 , 0.21474391,
       0.06079913], dtype=float32), array([0.02918336, 0.09843406, 0.06994466, ..., 0.08505121, 0.04785391,
       0.0012267 ], dtype=float32), array([0.01256198, 0.38367704, 0.11636376, ..., 0.01422517, 0.09370642,
       0.00537209], dtype=float32),

### Combining features

In [5]:
print(numerical_data.head(100))
# numerical_data['date']=numerical_data['date'].dt.month

         date measurement     0     1     2     3     4     5     6     7  \
0  2022-02-01    AMB_TEMP    16    16    16  16.4  16.4  16.7  16.8  16.9   
1  2022-02-01         CH4     2  2.02  2.05  2.04  2.07  2.09  2.08  2.11   
2  2022-02-01          CO  0.24  0.25  0.23   0.2   0.2  0.21  0.21  0.28   
3  2022-02-01        NMHC  0.03  0.04  0.02  0.02  0.03  0.03  0.02  0.05   
4  2022-02-01          NO   0.2   0.6   0.6   0.6   0.5   0.6   0.5     1   
..        ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   
95 2022-02-06         NO2   7.5   4.7   4.2     5   4.8   4.8     7    10   
96 2022-02-06         NOx   8.4   5.1   4.7   5.6   5.4   5.2   7.5  10.7   
97 2022-02-06          O3  46.5    51  50.9  48.5  47.2  45.2  40.9  36.9   
98 2022-02-06        PM10    25    28    26    29    23    16    17    11   
99 2022-02-06       PM2.5    22    20    21    21    20    13     9     9   

    ...    14    15    16    17    18    19    20    21    22    23  
0   .

In [6]:
print(numerical_data.shape)

(1062, 26)


In [7]:
# Reshape numerical data to match image data
#drop the Station column
# print(numerical_data.head())
# numerical_data = numerical_data.drop(columns=['Station'])
numerical_data = numerical_data.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
numerical_data['hour'] = numerical_data['hour'].astype(int)
numerical_data=numerical_data.pivot(index=['date','hour'],columns='measurement',values='value').reset_index()
# numerical_data['date']=numerical_data['date'].dt.month
numerical_data['month']=numerical_data['date'].dt.month
print(numerical_data.head(100))
# Create combined features for each date and hour
combined_features = []
targets = []
#instead of each row, we take 18 rows at a time
for idx, row in numerical_data.iterrows():
    date = row['date'].date()
    hour = row['hour']
    numerical_features = row.drop(['date']).values
    # print(hour)
    # print(numerical_features)
    # print(image_features[date][23])
    img_features = image_features[date][hour]
    print(img_features)
    print(numerical_features)
    #we can consider adding latitute and longitude as well to the input features
    # numerical_features=np.array(numerical_features).reshape(1,-1)
    # combined_feature=
    # combined_feature=np.array()
    combined_feature = np.concatenate((numerical_features, img_features),axis=None)
    combined_features.append(combined_feature)
    # targets.append(row['value'])  # Assuming the target is the value for that hour
    targets.append(row.drop(['date','hour','month']).values)  # Assuming the target is the value for that hour

combined_features = np.array(combined_features)
targets = np.array(targets)


measurement       date  hour AMB_TEMP   CH4    CO  NMHC   NO  NO2  NOx    O3  \
0           2022-02-01     0       16     2  0.24  0.03  0.2  7.1  7.4  37.9   
1           2022-02-01     1       16  2.02  0.25  0.04  0.6  6.2  6.8  36.9   
2           2022-02-01     2       16  2.05  0.23  0.02  0.6  5.6  6.3  35.8   
3           2022-02-01     3     16.4  2.04   0.2  0.02  0.6    4  4.6  37.5   
4           2022-02-01     4     16.4  2.07   0.2  0.03  0.5  3.7  4.3  37.3   
..                 ...   ...      ...   ...   ...   ...  ...  ...  ...   ...   
95          2022-02-04    23     13.5  2.02  0.28  0.01  0.6    9  9.6  36.6   
96          2022-02-05     0     13.5  2.01  0.25  0.01  0.4  7.1  7.6  39.1   
97          2022-02-05     1     13.6  2.01  0.24     0  0.8  5.4  6.3  40.6   
98          2022-02-05     2     13.6  2.02  0.24     0  0.8  4.6  5.4  41.7   
99          2022-02-05     3     13.5  2.02  0.24     0  0.9  3.6  4.5  43.3   

measurement  ... PM2.5 RAINFALL  RH  SO

In [8]:
print(numerical_data)
# targets=np.array(targets).reshape(-1)
print(targets)
# targets=targets[:,np.newaxis]
# targets=targets.squeeze()
# targets.reshape(-1,1)
print(targets.shape)

measurement       date  hour AMB_TEMP   CH4    CO  NMHC   NO   NO2   NOx  \
0           2022-02-01     0       16     2  0.24  0.03  0.2   7.1   7.4   
1           2022-02-01     1       16  2.02  0.25  0.04  0.6   6.2   6.8   
2           2022-02-01     2       16  2.05  0.23  0.02  0.6   5.6   6.3   
3           2022-02-01     3     16.4  2.04   0.2  0.02  0.6     4   4.6   
4           2022-02-01     4     16.4  2.07   0.2  0.03  0.5   3.7   4.3   
...                ...   ...      ...   ...   ...   ...  ...   ...   ...   
1411        2022-03-31    19     20.1     2  0.41  0.08  1.3  19.3  20.6   
1412        2022-03-31    20     19.9  2.01   0.4  0.05  1.1    17  18.2   
1413        2022-03-31    21     19.2  2.01  0.38  0.03  1.1  12.3  13.4   
1414        2022-03-31    22     18.8  2.01  0.36  0.02  0.9  12.7  13.7   
1415        2022-03-31    23     18.3     2  0.29     0  0.8   9.1    10   

measurement    O3  ... PM2.5 RAINFALL  RH  SO2   THC WD_HR WIND_DIREC  \
0            3

In [9]:
print(numerical_data.shape)
print(numerical_data.head(100))

(1416, 21)
measurement       date  hour AMB_TEMP   CH4    CO  NMHC   NO  NO2  NOx    O3  \
0           2022-02-01     0       16     2  0.24  0.03  0.2  7.1  7.4  37.9   
1           2022-02-01     1       16  2.02  0.25  0.04  0.6  6.2  6.8  36.9   
2           2022-02-01     2       16  2.05  0.23  0.02  0.6  5.6  6.3  35.8   
3           2022-02-01     3     16.4  2.04   0.2  0.02  0.6    4  4.6  37.5   
4           2022-02-01     4     16.4  2.07   0.2  0.03  0.5  3.7  4.3  37.3   
..                 ...   ...      ...   ...   ...   ...  ...  ...  ...   ...   
95          2022-02-04    23     13.5  2.02  0.28  0.01  0.6    9  9.6  36.6   
96          2022-02-05     0     13.5  2.01  0.25  0.01  0.4  7.1  7.6  39.1   
97          2022-02-05     1     13.6  2.01  0.24     0  0.8  5.4  6.3  40.6   
98          2022-02-05     2     13.6  2.02  0.24     0  0.8  4.6  5.4  41.7   
99          2022-02-05     3     13.5  2.02  0.24     0  0.9  3.6  4.5  43.3   

measurement  ... PM2.5 RAINF

In [10]:
print(combined_features.shape)

(1416, 2068)


In [11]:
print(combined_features.shape)
print(combined_features)
#convert all fields to float
#there are some fields that are not float
def to_float_with_nan(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

# Vectorize the function to apply it to the entire array
vectorized_to_float_with_nan = np.vectorize(to_float_with_nan)

# Replace garbage values with np.nan and convert to float
combined_features = vectorized_to_float_with_nan(combined_features).astype(np.float32)
targets=vectorized_to_float_with_nan(targets).astype(np.float32)

(1416, 2068)
[[0 '16' '2' ... 0.12950025498867035 0.017287785187363625
  0.0006822834257036448]
 [1 '16' '2.02' ... 0.1180066466331482 0.10802081227302551
  0.010468045249581337]
 [2 '16' '2.05' ... 0.08011416345834732 0.07610244303941727
  0.03191404417157173]
 ...
 [21 '19.2' '2.01' ... 0.026038488373160362 0.16109630465507507
  0.10558760911226273]
 [22 '18.8' '2.01' ... 0.011120679788291454 0.0527450256049633
  0.022969327867031097]
 [23 '18.3' '2' ... 0.06276505440473557 0.059047114104032516
  0.10566335916519165]]


In [12]:
print(combined_features)
print(targets.shape)

[[0.0000000e+00 1.6000000e+01 2.0000000e+00 ... 1.2950025e-01
  1.7287785e-02 6.8228343e-04]
 [1.0000000e+00 1.6000000e+01 2.0200000e+00 ... 1.1800665e-01
  1.0802081e-01 1.0468045e-02]
 [2.0000000e+00 1.6000000e+01 2.0500000e+00 ... 8.0114163e-02
  7.6102443e-02 3.1914044e-02]
 ...
 [2.1000000e+01 1.9200001e+01 2.0100000e+00 ... 2.6038488e-02
  1.6109630e-01 1.0558761e-01]
 [2.2000000e+01 1.8799999e+01 2.0100000e+00 ... 1.1120680e-02
  5.2745026e-02 2.2969328e-02]
 [2.3000000e+01 1.8299999e+01 2.0000000e+00 ... 6.2765054e-02
  5.9047114e-02 1.0566336e-01]]
(1416, 18)


In [13]:
print("Checking for NaN and infinite values in combined_features and targets...")
print("NaNs in combined_features:", np.isnan(combined_features).sum())
print("Infinite values in combined_features:", np.isinf(combined_features).sum())
print("NaNs in targets:", np.isnan(targets).sum())
print("Infinite values in targets:", np.isinf(targets).sum())

Checking for NaN and infinite values in combined_features and targets...
NaNs in combined_features: 0
Infinite values in combined_features: 0
NaNs in targets: 0
Infinite values in targets: 0


In [ ]:
# Replace NaNs and infinite values in combined_features and targets with zeros
combined_features = np.nan_to_num(combined_features, nan=0.0, posinf=0.0, neginf=0.0)
targets = np.nan_to_num(targets, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
# Replace NaNs in combined_features with column means
print("Replacing NaNs in combined_features with column means...")
col_mean_combined = np.nanmean(combined_features, axis=0)
inds_combined = np.where(np.isnan(combined_features))
combined_features[inds_combined] = np.take(col_mean_combined, inds_combined[1])

In [ ]:
# Replace NaNs in targets with column means
print("Replacing NaNs in targets with column means...")
col_mean_targets = np.nanmean(targets, axis=0)
inds_targets = np.where(np.isnan(targets))
targets[inds_targets] = np.take(col_mean_targets, inds_targets[1])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
combined_features[:,:20]=scaler.fit_transform(combined_features[:,:20])

### Model Design

In [14]:
from copy import deepcopy

print(combined_features.shape)
print(targets.shape)
targets=vectorized_to_float_with_nan(targets).astype(np.float32)

(1416, 2068)
(1416, 18)


In [32]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define custom dataset
class MultimodalDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Split data into train, validation, test sets
train_size = int(0.7 * len(combined_features))
val_size = int(0.1 * len(combined_features))
test_size = len(combined_features) - train_size - val_size

train_dataset = MultimodalDataset(combined_features[:train_size], targets[:train_size])
val_dataset = MultimodalDataset(combined_features[train_size:train_size + val_size], targets[train_size:train_size + val_size])
test_dataset = MultimodalDataset(combined_features[train_size + val_size:], targets[train_size + val_size:])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the multimodal neural network
class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()
        self.lstm_num = nn.LSTM(input_size=20, hidden_size=128, num_layers=1, batch_first=True)
        self.fc_num = nn.Linear(128, 64)  # Output from LSTM is 128, so input to fc is 128
        
        self.fc1_img = nn.Linear(2048, 128)  # ResNet output size
        self.fc2_img = nn.Linear(128, 64)
        
        self.fc1_combined = nn.Linear(128, 64)
        self.fc2_combined = nn.Linear(64, 18)  # Output size (18 for regression)
        
    def forward(self, x):
        x_num = x[:, :20].unsqueeze(1)  # First 20 features are numerical, add a dimension for LSTM (batch_size, seq_length, input_size)
        x_img = x[:, 20:]  # The rest are image features
        
        lstm_out, _ = self.lstm_num(x_num)  # Pass through LSTM
        x_num = lstm_out[:, -1, :]  # Take the output of the last time step
        x_num = torch.relu(self.fc_num(x_num))  # Pass through fully connected layer
        
        x_img = torch.relu(self.fc1_img(x_img))
        x_img = torch.relu(self.fc2_img(x_img))
        
        x_combined = torch.cat((x_num, x_img), dim=1)
        x_combined = torch.relu(self.fc1_combined(x_combined))
        x_combined = self.fc2_combined(x_combined)
        
        return x_combined
    

# Instantiate and train the model
model = MultimodalNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(200):
    print(f'Epoch {epoch+1}')
    model.train()
    running_loss = 0.0
    for i, (inputs, targets_) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets_)
        if torch.isnan(loss):
            print("NaN loss detected")
            print("Inputs: ", inputs)
            print("Outputs: ", outputs)
            print("Targets: ", targets_)
            break
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1
Epoch 1, Loss: 3518.1004126764115
Epoch 2
Epoch 2, Loss: 2093.7645401493196
Epoch 3
Epoch 3, Loss: 1075.271013813634
Epoch 4
Epoch 4, Loss: 678.8572584582913
Epoch 5
Epoch 5, Loss: 418.7157464796497
Epoch 6
Epoch 6, Loss: 331.83946621802545
Epoch 7
Epoch 7, Loss: 282.6070805211221
Epoch 8
Epoch 8, Loss: 260.44730426419164
Epoch 9
Epoch 9, Loss: 247.19626863541143
Epoch 10
Epoch 10, Loss: 238.10616819320185
Epoch 11
Epoch 11, Loss: 227.94239954794608
Epoch 12
Epoch 12, Loss: 216.93775103169102
Epoch 13
Epoch 13, Loss: 200.94499649540072
Epoch 14
Epoch 14, Loss: 176.20476827313823
Epoch 15
Epoch 15, Loss: 144.31233609107232
Epoch 16
Epoch 16, Loss: 115.03858160203502
Epoch 17
Epoch 17, Loss: 89.58078015235162
Epoch 18
Epoch 18, Loss: 73.92218878961378
Epoch 19
Epoch 19, Loss: 66.29549112627583
Epoch 20
Epoch 20, Loss: 63.688599001976755
Epoch 21
Epoch 21, Loss: 62.174610876267955
Epoch 22
Epoch 22, Loss: 59.19832807971585
Epoch 23
Epoch 23, Loss: 58.85777282714844
Epoch 24
Epoch 

In [16]:
class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()
        self.fc1_num = nn.Linear(20, 128)  # 20 numerical features
        self.fc2_num = nn.Linear(128, 64)

        self.fc1_img = nn.Linear(2048, 128)  # ResNet output size
        self.fc2_img = nn.Linear(128, 64)

        self.fc1_combined = nn.Linear(128, 64)
        self.fc2_combined = nn.Linear(64, 18)  # Output size (18 for regression)

    def forward(self, x):
        x_num = x[:, :20]  # First 20 features are numerical
        x_img = x[:, 20:]  # The rest are image features

        x_num = torch.relu(self.fc1_num(x_num))
        x_num = torch.relu(self.fc2_num(x_num))

        x_img = torch.relu(self.fc1_img(x_img))
        x_img = torch.relu(self.fc2_img(x_img))

        x_combined = torch.cat((x_num, x_img), dim=1)
        x_combined = torch.relu(self.fc1_combined(x_combined))
        x_combined = self.fc2_combined(x_combined)

        return x_combined


### Evaluation

In [33]:
# Evaluation
model.eval()
total_loss = 0.0
with torch.no_grad():
    for inputs, targets_ in val_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets_)
        total_loss += loss.item()
print(f'Validation Loss: {total_loss/len(val_loader)}')
# Deployment
# Implement a system to gather new data, preprocess it, and use the trained model to make predictions


Validation Loss: 28.283308029174805


### Prediction

In [51]:
# Convert relevant columns to numeric type
numerical_data = numerical_data.apply(pd.to_numeric, errors='coerce')

# Calculate the mean of each measurement for every month, excluding NaN values
monthly_means = numerical_data.groupby('month').mean()

print("Monthly Means:")
print(monthly_means)


Monthly Means:
measurement          date  hour   AMB_TEMP       CH4        CO      NMHC  \
month                                                                      
2            1.644840e+18  11.5  16.598065  2.068741  0.416432  0.088516   
3            1.647389e+18  11.5  21.454582  2.094030  0.500312  0.147449   

measurement        NO        NO2        NOx         O3       PM10      PM2.5  \
month                                                                          
2            2.724474  15.562913  18.330480  30.393874  15.369925   9.347368   
3            4.611580  18.784741  23.442507  32.295317  26.089796  15.821769   

measurement  RAINFALL         RH       SO2       THC       WD_HR  WIND_DIREC  \
month                                                                          
2            0.387556  82.953869  1.072864  2.156867  128.637854  131.487332   
3            0.271370  74.063342  1.542241  2.241221  148.944744  141.371467   

measurement  WIND_SPEED     WS_HR  
mo

In [62]:
import os
import pandas as pd
from PIL import Image,UnidentifiedImageError
import numpy as np
#the values are a bit off for tucheng
#maybe try with banqiao itself
# Path to the folder containing subfolders for each day
image_folder = './input/20220202'

# Load numerical data
numerical_input = pd.read_csv('./input/Tucheng_2022.csv')
#drop the Station column
numerical_input = numerical_input.drop(columns=['Station'])
# Ensure the 'date' column in numerical_data is in datetime format
print(numerical_input.head())
numerical_input['date'] = pd.to_datetime(numerical_input['date'],format="%d-%m-%Y %H:%M",dayfirst=True)

# Function to load images for a specific date
def load_images_for_date(date):
    date_str = date.strftime('%Y%m%d')
    images = []
    for hour in range(24):
        img_name=f"005-{date_str}{hour:02d}00.jpg"
        img_path = os.path.join(image_folder, img_name)
        try: 
            if os.path.exists(img_path):
                with Image.open(img_path) as img:
                    images.append(img.copy())
            else:
                images.append(None)
        except(OSError,UnidentifiedImageError):
            images.append(None)
    return images

# Create a dictionary to store images by date
image_input = {date: load_images_for_date(date) for date in numerical_input['date'].dt.date.unique()}

               date measurement      0      1      2      3      4      5  \
0  02-02-2022 00:00    AMB_TEMP  16.40  16.40  16.50  16.50  16.60  16.60   
1  02-02-2022 00:00         CH4   2.06   2.07   2.11   2.11   2.11   2.10   
2  02-02-2022 00:00          CO   0.24   0.27   0.29   0.29   0.30   0.31   
3  02-02-2022 00:00        NMHC   0.05   0.03   0.04   0.04   0.04   0.04   
4  02-02-2022 00:00          NO   0.40   0.20   0.40   0.50   0.50   0.30   

       6      7  ...     14     15     16     17     18     19     20     21  \
0  16.80  16.90  ...  19.80  18.30  17.60  17.50  17.80  17.80  17.70  17.80   
1   2.10   2.08  ...   2.11   2.13   2.15   2.14   2.13   2.10   2.10   2.09   
2   0.30   0.30  ...   0.35   0.33   0.36   0.43   0.42   0.32   0.31   0.31   
3   0.04   0.05  ...   0.08   0.07   0.09   0.11   0.11   0.07   0.06   0.06   
4   0.40   0.50  ...   0.80   0.60   0.60   0.60   0.60   0.60   0.50   0.50   

      22     23  
0  18.00  17.90  
1   2.09   2.16  
2 

In [63]:
import torchvision.transforms as transforms
from torchvision.models import resnet50,ResNet50_Weights
import torch

# Preprocess images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load pretrained ResNet model
resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final classification layer

# Extract features from images
def extract_image_features(images):
    features = []
    for img in images:
        if img is not None:
            img = transform(img).unsqueeze(0)  # Add batch dimension
            with torch.no_grad():
                feature = resnet(img).squeeze()  # Remove batch dimension
            features.append(feature.numpy())
        else:
            features.append(np.zeros(2048))  # Handle missing images
    return features

# Extract image features for each date
image_input_features = {date: extract_image_features(images) for date, images in image_input.items()}
#free up space
del resnet

In [64]:
print(image_input_features)

{datetime.date(2022, 2, 2): [array([0.01280319, 0.08944495, 0.07654931, ..., 0.08754218, 0.05505829,
       0.00961756], dtype=float32), array([0.06909723, 0.33076748, 0.125428  , ..., 0.08345933, 0.01354523,
       0.22965688], dtype=float32), array([0.00976614, 0.08134273, 0.11898661, ..., 0.03633262, 0.03346932,
       0.03950009], dtype=float32), array([0.11725301, 0.02975344, 0.03065366, ..., 0.0320784 , 0.04232978,
       0.03517786], dtype=float32), array([0.13053368, 0.03675611, 0.21286552, ..., 0.10800242, 0.0264472 ,
       0.11536852], dtype=float32), array([0.01566086, 0.08984747, 0.0494442 , ..., 0.03883245, 0.11160567,
       0.03982995], dtype=float32), array([0.03857847, 0.05730736, 0.00486739, ..., 0.04852929, 0.04401107,
       0.        ], dtype=float32), array([0.04490849, 0.05868247, 0.27703187, ..., 0.0865263 , 0.01858796,
       0.16979384], dtype=float32), array([0.03630832, 0.04972351, 0.09669606, ..., 0.02765603, 0.05833967,
       0.16991179], dtype=float32),

In [65]:
# Reshape numerical data to match image data
numerical_input = numerical_input.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
numerical_input['hour'] = numerical_input['hour'].astype(int)
numerical_input=numerical_input.pivot(index=['date','hour'],columns='measurement',values='value').reset_index()
numerical_input['month']=numerical_input['date'].dt.month
#replace values with monthly means
def replace_with_monthly_means(row, monthly_means):
    month = row['month']
    if month in monthly_means.index:
        monthly_mean_values = monthly_means.loc[month]
        for col in monthly_mean_values.index:
            if col!='month' and col!='hour' and col!='date':
                row[col] = monthly_mean_values[col]
    return row
numerical_input=numerical_input.apply(replace_with_monthly_means, args=(monthly_means,), axis=1)
print(numerical_input.head(100))
# Create combined features for each date and hour
combined_input_features = []
targets_input = []
#instead of each row, we take 18 rows at a time
for idx, row in numerical_input.iterrows():
    date = row['date'].date()
    hour = row['hour']
    numerical_features = row.drop(['date']).values
    img_features = image_input_features[date][hour]
    print(img_features)
    print(numerical_features)
    combined_feature = np.concatenate((numerical_features, img_features),axis=None)
    combined_input_features.append(combined_feature)
    targets_input.append(row.drop(['date','hour','month']).values)  # Assuming the target is the value for that hour

combined_input_features = np.array(combined_input_features)
targets_input = np.array(targets_input)


measurement       date  hour   AMB_TEMP       CH4        CO      NMHC  \
0           2022-02-02     0  16.598065  2.068741  0.416432  0.088516   
1           2022-02-02     1  16.598065  2.068741  0.416432  0.088516   
2           2022-02-02     2  16.598065  2.068741  0.416432  0.088516   
3           2022-02-02     3  16.598065  2.068741  0.416432  0.088516   
4           2022-02-02     4  16.598065  2.068741  0.416432  0.088516   
5           2022-02-02     5  16.598065  2.068741  0.416432  0.088516   
6           2022-02-02     6  16.598065  2.068741  0.416432  0.088516   
7           2022-02-02     7  16.598065  2.068741  0.416432  0.088516   
8           2022-02-02     8  16.598065  2.068741  0.416432  0.088516   
9           2022-02-02     9  16.598065  2.068741  0.416432  0.088516   
10          2022-02-02    10  16.598065  2.068741  0.416432  0.088516   
11          2022-02-02    11  16.598065  2.068741  0.416432  0.088516   
12          2022-02-02    12  16.598065  2.068741  

In [66]:
print(combined_input_features.shape)
print(combined_input_features)
#convert all fields to float
#there are some fields that are not float
def to_float_with_nan(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

# Vectorize the function to apply it to the entire array
vectorized_to_float_with_nan = np.vectorize(to_float_with_nan)

# Replace garbage values with np.nan and convert to float
combined_input_features = vectorized_to_float_with_nan(combined_input_features).astype(np.float32)
targets_input=vectorized_to_float_with_nan(targets_input).astype(np.float32)

(24, 2068)
[[0 16.598065476190477 2.0687406296851574 ... 0.08754218369722366
  0.05505828931927681 0.009617557749152184]
 [1 16.598065476190477 2.0687406296851574 ... 0.08345932513475418
  0.013545233756303787 0.2296568751335144]
 [2 16.598065476190477 2.0687406296851574 ... 0.0363326221704483
  0.033469315618276596 0.03950009122490883]
 ...
 [21 16.598065476190477 2.0687406296851574 ... 0.09349971264600754
  0.04414798691868782 0.06249909847974777]
 [22 16.598065476190477 2.0687406296851574 ... 0.07206287980079651
  0.014108873903751373 0.17843236029148102]
 [23 16.598065476190477 2.0687406296851574 ... 0.03136361762881279
  0.06723427027463913 0.0]]


In [67]:
print(combined_input_features)
print(targets_input.shape)

[[0.0000000e+00 1.6598066e+01 2.0687406e+00 ... 8.7542184e-02
  5.5058289e-02 9.6175577e-03]
 [1.0000000e+00 1.6598066e+01 2.0687406e+00 ... 8.3459325e-02
  1.3545234e-02 2.2965688e-01]
 [2.0000000e+00 1.6598066e+01 2.0687406e+00 ... 3.6332622e-02
  3.3469316e-02 3.9500091e-02]
 ...
 [2.1000000e+01 1.6598066e+01 2.0687406e+00 ... 9.3499713e-02
  4.4147987e-02 6.2499098e-02]
 [2.2000000e+01 1.6598066e+01 2.0687406e+00 ... 7.2062880e-02
  1.4108874e-02 1.7843236e-01]
 [2.3000000e+01 1.6598066e+01 2.0687406e+00 ... 3.1363618e-02
  6.7234270e-02 0.0000000e+00]]
(24, 18)


In [68]:
print("Checking for NaN and infinite values in combined_input_features and targets_input...")
print("NaNs in combined_input_features:", np.isnan(combined_input_features).sum())
print("Infinite values in combined_input_features:", np.isinf(combined_input_features).sum())
print("NaNs in targets_input:", np.isnan(targets_input).sum())
print("Infinite values in targets_input:", np.isinf(targets_input).sum())

Checking for NaN and infinite values in combined_input_features and targets_input...
NaNs in combined_input_features: 0
Infinite values in combined_input_features: 0
NaNs in targets_input: 0
Infinite values in targets_input: 0


In [69]:
# Replace NaNs and infinite values in combined_input_features and targets_input with zeros
combined_input_features = np.nan_to_num(combined_input_features, nan=0.0, posinf=0.0, neginf=0.0)
targets_input = np.nan_to_num(targets_input, nan=0.0, posinf=0.0, neginf=0.0)

In [70]:
# Replace NaNs in combined_input_features with column means
print("Replacing NaNs in combined_input_features with column means...")
col_mean_combined = np.nanmean(combined_input_features, axis=0)
inds_combined = np.where(np.isnan(combined_input_features))
combined_input_features[inds_combined] = np.take(col_mean_combined, inds_combined[1])

Replacing NaNs in combined_input_features with column means...


In [71]:
# Replace NaNs in targets_input with column means
print("Replacing NaNs in targets_input with column means...")
col_mean_targets_input = np.nanmean(targets_input, axis=0)
inds_targets_input = np.where(np.isnan(targets_input))
targets_input[inds_targets_input] = np.take(col_mean_targets_input, inds_targets_input[1])

Replacing NaNs in targets_input with column means...


In [72]:
print(numerical_input)
#convert all the numerical input

measurement       date  hour   AMB_TEMP       CH4        CO      NMHC  \
0           2022-02-02     0  16.598065  2.068741  0.416432  0.088516   
1           2022-02-02     1  16.598065  2.068741  0.416432  0.088516   
2           2022-02-02     2  16.598065  2.068741  0.416432  0.088516   
3           2022-02-02     3  16.598065  2.068741  0.416432  0.088516   
4           2022-02-02     4  16.598065  2.068741  0.416432  0.088516   
5           2022-02-02     5  16.598065  2.068741  0.416432  0.088516   
6           2022-02-02     6  16.598065  2.068741  0.416432  0.088516   
7           2022-02-02     7  16.598065  2.068741  0.416432  0.088516   
8           2022-02-02     8  16.598065  2.068741  0.416432  0.088516   
9           2022-02-02     9  16.598065  2.068741  0.416432  0.088516   
10          2022-02-02    10  16.598065  2.068741  0.416432  0.088516   
11          2022-02-02    11  16.598065  2.068741  0.416432  0.088516   
12          2022-02-02    12  16.598065  2.068741  

In [73]:
#predict using the model
# Evaluation
input_loader=DataLoader(MultimodalDataset(combined_input_features, targets_input), batch_size=24)
model.eval()
total_loss = 0.0
measures=["AMB_TEMP","CH4","CO","NHMC","NO","NO2","NOx","O3","PM10","PM2.5","RAINFALL","RH","SO2","THC","WD_HR","WIND_DIREC","WIND_SPEED","WS_HR"]
with torch.no_grad():
    for inputs, targets_ in input_loader:
        print(inputs.size())
        outputs = model(inputs)
        print(outputs.size())
        for i in range(24):
            print(f'Hour {i}')
            for j in range(18):
                print(f'{measures[j]}: {outputs[i][j]}')
        loss = criterion(outputs, targets_)
        total_loss += loss.item()
print(f'Test Loss: {total_loss/len(input_loader)}')
# Deployment
# Implement a system to gather new data, preprocess it, and use the trained model to make predictions


torch.Size([24, 2068])
torch.Size([24, 18])
Hour 0
AMB_TEMP: 16.725351333618164
CH4: 1.7712658643722534
CO: 0.32746729254722595
NHMC: 0.3578287363052368
NO: 2.1859729290008545
NO2: 12.099477767944336
NOx: 14.413651466369629
O3: 28.922285079956055
PM10: 14.107145309448242
PM2.5: 8.332143783569336
RAINFALL: -0.39566129446029663
RH: 80.78858184814453
SO2: 1.195578932762146
THC: 2.0032308101654053
WD_HR: 122.22071075439453
WIND_DIREC: 121.52115631103516
WIND_SPEED: 1.3791111707687378
WS_HR: 1.338428258895874
Hour 1
AMB_TEMP: 18.233930587768555
CH4: 1.9240227937698364
CO: 0.29625147581100464
NHMC: 0.275873601436615
NO: 2.363924026489258
NO2: 13.511329650878906
NOx: 16.215702056884766
O3: 26.29788589477539
PM10: 13.726686477661133
PM2.5: 6.841663837432861
RAINFALL: -0.6231671571731567
RH: 78.47977447509766
SO2: 1.3962466716766357
THC: 1.9888297319412231
WD_HR: 118.595703125
WIND_DIREC: 118.02680206298828
WIND_SPEED: 1.4411466121673584
WS_HR: 1.50386643409729
Hour 2
AMB_TEMP: 19.0696811676025